In [8]:
import sys
sys.path.append('../')

In [9]:
import libs.db
import libs.prep_df
import pandas as pd
import sqlite3

In [10]:
sql_file = "sp10_not_normalized.db"
conn = libs.db.create_connection(sql_file)
df = libs.db.df_from_db(conn)
libs.db.close_connection(conn)
df.head()

,id,symbol,timestamp,close
0,1,AAPL,2017-01-03T09:00:00+00:00,116.05
1,2,AAPL,2017-01-03T10:00:00+00:00,116.02
2,3,AAPL,2017-01-03T11:00:00+00:00,116.28
3,4,AAPL,2017-01-03T12:00:00+00:00,116.02
4,5,AAPL,2017-01-03T13:00:00+00:00,116.20


In [11]:
norm_df = libs.prep_df.normalize_df(df)

In [12]:
tickers = libs.db.get_tickers("sp10.csv")
print(tickers)

['AAPL', 'MSFT', 'AMZN', 'FB', 'GOOGL', 'TSLA', 'BRK.B', 'NVDA', 'JPM', 'JNJ']


In [13]:
num_ticks = 60
ticker_shifted_df = pd.DataFrame()
for ticker in tickers:
    ticker_shifted_df[ticker] = norm_df[ticker].copy()
    for i in range(1,num_ticks):
        name = f"{ticker}-{i}"
        ticker_shifted_df[name] = norm_df[ticker].shift(i)
ticker_shifted_df.shape

(17432, 600)

In [15]:
ticker_returns_df_list = []
for ticker in tickers:
    name = f"{ticker}_24_Return"
    returns_df = pd.DataFrame()
    returns_df[name] = norm_df[ticker].pct_change(periods=16).shift(-16)
    ticker_returns_df_list.append(returns_df)
len(ticker_returns_df_list)

10

In [19]:
a = [1,2,3,4,5]
b = [*a,6]
b

[1, 2, 3, 4, 5, 6]

In [23]:
pctchange_df = norm_df.pct_change()
pctchange_df = pd.concat([ticker_shifted_df, *ticker_returns_df_list], axis=1)
pctchange_df.dropna(inplace=True)
pctchange_df.describe()

,AAPL,AAPL-1,AAPL-2,AAPL-3,AAPL-4,AAPL-5,AAPL-6,AAPL-7,AAPL-8,AAPL-9,...,AAPL_24_Return,MSFT_24_Return,AMZN_24_Return,FB_24_Return,GOOGL_24_Return,TSLA_24_Return,BRK.B_24_Return,NVDA_24_Return,JPM_24_Return,JNJ_24_Return
count,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000,17357.00000,17357.000000,17357.000000,17357.000000,17357.000000,...,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000,17357.000000
mean,194.631383,194.630479,194.629567,194.628659,194.627756,194.62683,194.625894,194.624892,194.623911,194.622922,...,0.000847,0.001488,0.001890,0.001168,0.001153,0.002464,0.000572,0.002276,0.000697,0.000395
std,72.162164,72.163026,72.163896,72.164763,72.165625,72.16651,72.167402,72.168366,72.169309,72.170261,...,0.029545,0.017823,0.055956,0.021328,0.017369,0.046288,0.013279,0.029485,0.018539,0.012852
min,104.050000,104.050000,104.050000,104.050000,104.050000,104.05000,104.050000,104.050000,104.050000,104.050000,...,-0.738819,-0.441069,-0.873505,-0.219802,-0.127222,-0.795029,-0.108916,-0.198644,-0.149697,-0.116202
25%,143.770000,143.770000,143.770000,143.770000,143.770000,143.77000,143.770000,143.770000,143.770000,143.770000,...,-0.006999,-0.006024,-0.007416,-0.008167,-0.006512,-0.016133,-0.005638,-0.012388,-0.007496,-0.005223
50%,174.850000,174.850000,174.850000,174.850000,174.850000,174.85000,174.850000,174.850000,174.850000,174.850000,...,0.001553,0.001346,0.001670,0.001119,0.001521,0.001569,0.000671,0.002406,0.000350,0.000567
75%,218.340000,218.340000,218.340000,218.340000,218.340000,218.34000,218.340000,218.340000,218.340000,218.340000,...,0.010504,0.009563,0.011091,0.011469,0.009534,0.020188,0.006999,0.018103,0.008951,0.006675
max,513.890000,513.890000,513.890000,513.890000,513.890000,513.89000,513.890000,513.890000,513.890000,513.890000,...,0.176167,0.808791,6.868773,0.155632,0.116159,0.316360,0.118387,0.225417,0.180847,0.105285
